# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Load the datasdet dair-air/emotion and explore the data

In [1]:
from datasets import load_dataset

ds = load_dataset("dair-ai/emotion", "split")
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [2]:
import random

# print some random featues and the labels
print("Features:")
indices = random.sample(range(len(ds["train"])), 10)
for i in indices:
    print("{} : {}".format(ds["train"]['text'][i], ds["train"]['label'][i]))

print("\nLabels: {}".format(ds["train"].features["label"].names))

Features:
i feel a divine calling to become an lsd chemist the pub shroomery message board link href smarty templates css www : 1
i keep in the fridge and take out whenever i feel too hot spray on my face and it cools me down immediately let alone it smells like oranges : 2
i feel sure a new necklace will come from this afternoon of beach combing : 1
i feel like i want to make something but the house is so messy and i am still finishing up christmas gift knitting : 0
i was feeling determined it didnt take long for me to start nomming on naughty stuff again : 1
i had to be transgender and this very brainwashing attempt is now making me feel so horrible as though im trying to deceive physicians here in germany into believing that i am intersex : 0
im taking this to heart and feel that the gentle age of is the perfect time and place to engage in some fearlessness and wholesale change : 2
i still second guess myself and still have a terrible time making definitive decisions but there are c

In [3]:
# create data structures for further processing

# names of the splits
splits=list(ds.keys())
# number of classes
num_classes=len(ds["train"].features["label"].names)

# Dictionairies to translate between label string and label number
id2label = dict(zip(range(num_classes), ds['train'].features['label'].names))
label2id = dict(zip(ds['train'].features['label'].names, range(num_classes)))
print(id2label)
print(label2id)

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Use GPT-2 as a small base model
# Create a variant with classification head
device = torch.accelerator.current_accelerator().type if hasattr(torch, "accelerator") else "cuda"
model_id = "openai-community/gpt2"
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, 
    num_labels=num_classes,
    id2label=id2label,
    label2id=label2id,
    device_map=device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Add tokens to the dataset
tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True
    )

for param in model.base_model.parameters():
    param.requires_grad = False

# Add the padding token which is missing in GPT-2
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = model.config.eos_token_id
    print("Padding token: {}".format(tokenizer.pad_token))

# metric function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Padding token: [PAD]


In [5]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import os

temp_path = "/tmp"
save_path = "./data"

model_name = "gpt2_classification"
checkpoint_dir = os.path.join(temp_path, model_name)
save_dir_base = os.path.join(save_path, model_name)

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir=checkpoint_dir,
        learning_rate=2e-3,
        per_device_train_batch_size=100,
        per_device_eval_batch_size=100,
        num_train_epochs=5,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.199998,0.539000
2,No log,1.142866,0.575000
3,No log,1.103300,0.583000
4,1.224700,1.121378,0.590000
5,1.224700,1.089682,0.585000


TrainOutput(global_step=800, training_loss=1.1781553268432616, metrics={'train_runtime': 71.2181, 'train_samples_per_second': 1123.31, 'train_steps_per_second': 11.233, 'total_flos': 2309089289011200.0, 'train_loss': 1.1781553268432616, 'epoch': 5.0})

In [6]:
# Evaluate the model
original_performance=trainer.evaluate()
print(original_performance)

model.save_pretrained(save_dir_base, save_embedding_layers=True)

{'eval_loss': 1.0896823406219482, 'eval_accuracy': 0.585, 'eval_runtime': 1.3099, 'eval_samples_per_second': 1526.873, 'eval_steps_per_second': 15.269, 'epoch': 5.0}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [7]:
from peft import LoraConfig, TaskType, get_peft_model

torch.cuda.empty_cache()

# Use Lora for PEFT
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    task_type=TaskType.TOKEN_CLS,
    fan_in_fan_out=True,
)

# adding PEFT modifies the base model in-place
# so it should be saved for restoring the PEFT model later
model_lora = get_peft_model(model, peft_config)
model_lora.print_trainable_parameters()

model_name = "gpt2_classification_lora"
checkpoint_dir = os.path.join(temp_path, model_name)
save_dir = os.path.join(save_path, model_name)

trainable params: 594,432 || all params: 125,039,616 || trainable%: 0.4754


In [8]:
trainer_lora = Trainer(
    model=model_lora,
    args=TrainingArguments(
        output_dir=checkpoint_dir,
        learning_rate=2e-3,
        per_device_train_batch_size=100,
        per_device_eval_batch_size=100,
        num_train_epochs=5,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer_lora.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.239889,0.914500
2,No log,0.152287,0.932000
3,No log,0.136047,0.930500
4,0.251400,0.125707,0.933000
5,0.251400,0.121623,0.933000


/media/ssd1/venv/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/media/ssd1/venv/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/media/ssd1/venv/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/media/ssd1/venv/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/media/ssd1/venv/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has

TrainOutput(global_step=800, training_loss=0.20044589519500733, metrics={'train_runtime': 142.8262, 'train_samples_per_second': 560.121, 'train_steps_per_second': 5.601, 'total_flos': 2325225977856000.0, 'train_loss': 0.20044589519500733, 'epoch': 5.0})

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [9]:
# Saving the model

model_lora.save_pretrained(save_dir, save_embedding_layers=True)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [10]:
from peft import PeftModelForTokenClassification

# loading the model
model_base = AutoModelForSequenceClassification.from_pretrained(save_dir_base)
model_loaded = PeftModelForTokenClassification.from_pretrained(model_base, save_dir)

In [11]:
trainer_evaluate = Trainer(
    model=model_loaded,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis_lora_evaluate",
        per_device_train_batch_size=100,
        per_device_eval_batch_size=100,
        do_train=False,
        do_eval=True,
    ),
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

fine_tuned_performance=trainer_evaluate.evaluate()

In [12]:

print("Original Model:  ", original_performance)
print("Fine-Tuned Model:", fine_tuned_performance)

print("Original Model accurcy:   ", original_performance['eval_accuracy'])
print("Fine-Tuned Model accurcy: ", fine_tuned_performance['eval_accuracy'])

Original Model:   {'eval_loss': 1.0896823406219482, 'eval_accuracy': 0.585, 'eval_runtime': 1.3099, 'eval_samples_per_second': 1526.873, 'eval_steps_per_second': 15.269, 'epoch': 5.0}
Fine-Tuned Model: {'eval_loss': 0.121622733771801, 'eval_model_preparation_time': 0.002, 'eval_accuracy': 0.933, 'eval_runtime': 1.406, 'eval_samples_per_second': 1422.447, 'eval_steps_per_second': 14.224}
Original Model accurcy:    0.585
Fine-Tuned Model accurcy:  0.933
